In [2]:
from zipfile import ZipFile
import pandas as pd
import io

In [3]:
product_hierarchy = pd.read_csv('/home/tdjiang/github/stored_data/retail_sales_data/product_hierarchy.csv')
store_cities = pd.read_csv('/home/tdjiang/github/stored_data/retail_sales_data/store_cities.csv')

sales_zip = ZipFile('/home/tdjiang/github/stored_data/retail_sales_data/sales.csv.zip', 'r')
sales_data = sales_zip.read('sales.csv')
sales_bytes = io.BytesIO(sales_data)
sales_bytes.seek(0)
sales = pd.read_csv(sales_bytes, converters={'promo_bin_1':str,'promo_bin_2':str,'promo_discount_type_2':str})

In [14]:
sales.columns

Index(['product_id', 'store_id', 'date', 'sales', 'revenue', 'stock', 'price',
       'promo_type_1', 'promo_bin_1', 'promo_type_2', 'promo_bin_2',
       'promo_discount_2', 'promo_discount_type_2'],
      dtype='object')

In [13]:
sales[['promo_type_2','promo_bin_2','promo_discount_2','promo_discount_type_2']].fillna(0).value_counts()

promo_type_2  promo_bin_2  promo_discount_2  promo_discount_type_2
PR03                       0.0                                        19395757
PR02          verylow      16.0              PR02                        10679
              low          25.0              PR02                         9900
              verylow      20.0              PR02                         8920
PR04          high         33.0              PR01                         6833
PR02          verylow      20.0              PR04                         5971
PR01          veryhigh     50.0              PR03                         4478
PR02          verylow      10.0              PR02                         2999
              high         35.0              PR01                         1943
PR01          veryhigh     50.0              PR01                         1826
PR02          low          22.0              PR01                         1517
              verylow      16.0              PR01               